# Playing with the PAL algorithm.

There's something interesting: https://github.com/VitoChan1/PAL-on-Adder-DSE/blob/master/PAL.py

Summarizing the paper (PAL: An Active Learning Approach to the Multi-Objective Optimization Problem form the Puschel group)